# Data analysis relevent code

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pyupset as pyu
from numpy import NaN

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import os

## Nascent Transcript (Correlation with ChIP result)

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
ChIP=pd.read_table("~/PIRCh-seq/pirch_chip_intron_rc.txt",index_col=0)
exon_rc=ChIP.apply(lambda x:x.apply(lambda x:int(x.split(",")[0])),axis=1)

In [ ]:
for name in ["H3K4me1","H3K4me3","H3K27ac","H3K27me3","H3K9me3"]:
    PIRCh_result=pd.read_table("~/v65/limma_"+name+"_filter.txt",index_col=0)
    chip=exon_rc.loc[PIRCh_result.index].dropna()
    x=PIRCh_result.loc[chip.index]["logFC"]
    y=np.log2(chip["ChIP_"+name]+1)-np.log2(chip["ChIP_Input"]+1)
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy)
    plt.figure(figsize=[6,6])
    plt.xlim([-6,6])
    plt.ylim([-10,10])
    plt.scatter(x,y,c=z,s=2,alpha=0.3)
    A1, B1 = optimize.curve_fit(f_1, x, y)[0]  
    x1 = np.arange(-6, 6, 0.01)  
    y1 = A1*x1 + B1  
    plt.plot(x1, y1,'r--',linewidth=2,)
    plt.title(scipy.stats.pearsonr(x,y)[0]**2)
    plt.savefig("../V10/nascent_transcription/v65_"+name+"_chip.pdf")
    plt.savefig("../V10/nascent_transcription/v65_"+name+"_chip.png",dpi=300)
    plt.show()

## K-mean Cluster & T-SNE

In [ ]:
    home=("~/PIRCh-seq/v65/")
    data=pd.read_table("~/v65/rpkm.txt",index_col=0) # Read in Gene name
    FD=pd.DataFrame([],index=data.index,columns=map(lambda x:x.split("_")[1].split(".")[0],[x for x in os.listdir(home) if "filter" in x]))
    pvalue=pd.DataFrame([],index=data.index,columns=map(lambda x:x.split("_")[1].split(".")[0],[x for x in os.listdir(home) if "filter" in x]))
    aver=pd.DataFrame([],index=data.index,columns=map(lambda x:x.split("_")[1].split(".")[0],[x for x in os.listdir(home) if "filter" in x]))
    for a in [x for x in os.listdir(home) if "filter" in x]:
        temp=pd.read_table(home+a,index_col=0)
        FD[a.split("_")[1].split(".")[0]]=temp["logFC"]
        pvalue[a.split("_")[1].split(".")[0]]=temp["P.Value"]
        aver[a.split("_")[1].split(".")[0]]=temp["AveExpr"]
    FD=FD.fillna(value=0)
    pvalue=pvalue.fillna(value=1)
    aver=aver.fillna(value=0)
    a=aver[aver.mean(axis=1)>1]
    fdnc=FD.loc[a.loc[category[category!="Coding"].index].dropna().index]
    pnc=pvalue.loc[a.loc[category[category!="Coding"].index].dropna().index]
    def select(x):
        name=x.name
        result=[]
        for a in x.index[1:]:
            if (pnc.loc[name][a]<0.05) &(x[a]>0) :
                result.append(x[a])
            else:
                result.append(0)
        return pd.Series(result,index=x.index[1:])
    temp=fdnc.apply(select,axis=1)   ##(filter FD > 0 & P-value < 0.05)
    temp=temp[temp.sum(axis=1)!=0]
    model=TSNE(n_components=2,n_iter=10000,learning_rate=200,random_state=40)
    kmeans = KMeans(n_clusters=6,max_iter=10000)
    tsne=pd.DataFrame(model.fit_transform(temp),index=temp.index)
    kmeans.fit(tsne)
    cluster=pd.Series(kmeans.labels_,index=tsne.index)
    scatter(tsne,cluster)
    ovl=temp.copy()
    ovl[ovl>0]=1
    k=ovl.groupby(cluster)
    sns.heatmap((k.sum()/k.count()).iloc[:,[2,3,0,1,4,5]],cmap="Blues")
    plt.show()

In [ ]:
FD.to_csv("~/PIRCh-seq/v65/v65_fd.txt",sep="\t")
temp.to_csv("~/PIRCh-seq/v65/v65_enrich.txt",sep="\t")
ovl.to_csv("~/PIRCh-seq/v65/v65_bin.txt",sep="\t")
(k.sum()/k.count()).iloc[[0,4,1,5,3,2],[2,3,0,1,4,5]].to_csv("~/PIRCh-seq/v65/v65_stateinfo.txt",sep="\t")

## Overlap with GRID/ChIRP/CHART result

In [ ]:
cov=pd.read_table("~/PIRCh-seq/GRID_ChIRP_CHART/Peak.txt",index_col=0)
ovl=fdnc.iloc[:,[1,2,3,4,5]].loc[cov.index].dropna()
final_gene=ovl.apply(lambda x:scipy.stats.spearmanr(x,cov.loc[x.name])[0],axis=1).loc[cov.mean(axis=1).sort_values(ascending=False).index].dropna()
fig=plt.figure()
ax1=fig.add_subplot(1,2,1)
ax1=sns.heatmap(ovl.iloc[:,[2,3,0,1,4]].loc[final_gene.index].apply(lambda x:x.rank(),axis=1),cmap="Blues",cbar=False,yticklabels=True)
ax1=fig.add_subplot(1,2,2)
ax2=sns.heatmap(cov.iloc[:,[2,3,0,1,4]].loc[final_gene.index].apply(lambda x:x.rank(),axis=1),cmap="Blues",cbar=False,yticklabels=False)
plt.savefig("~/PIRCh-seq/GRID_ChIRP_CHART/Heatmap_overlap.pdf")
#plt.show()

## Nearby Coding gene expression

In [ ]:
location=[x.rstrip().split() for x in open("/seq/f_anno/mm9_nc_nearby_coding_1000K.txt")]
nearby=pd.Series([x[4:] for x in location if len(x)>4],index=[x[1] for x in location if len(x)>4 ])
express_data=aver[aver.mean(axis=1)>2]["H3"]
label=pd.read_table("~/PIRCh-seq/v65/v65_bin.txt",index_col=0)

In [ ]:
def test_func(data):
    t=express_data.ix[set([x for y in nearby.ix[data].dropna() for x in y])].dropna().values
    return t

e=cut10[[0,4,1,5,3,2]].apply(test_func)

fig, axes = plt.subplots(nrows=1, ncols=1)
n=axes.boxplot(e,notch=True,patch_artist=True)
for patch, color in zip(n['boxes'], colorst):
    patch.set_facecolor(color)
plt.savefig("~/PIRCh-seq/v65/v65/v65_near1000K_withnot.pdf")
plt.show()